In [13]:
import numpy as np
import cv2
import open3d as o3d
import time

# 读取深度图像
depth_img = cv2.imread("../img/1_depth.png", cv2.IMREAD_GRAYSCALE)


def depth_to_point_cloud(depth_img, k):
    # 获取深度图像的宽度和高度
    h, w = depth_img.shape

    # 初始化点云数组
    point_cloud = np.zeros((h * w, 3), dtype=np.float32)

    # 获取相机内参的逆矩阵
    k_inv = np.linalg.inv(k)

    for i in range(h):
        for j in range(w):
            # 如果像素深度值为0，则跳过该点
            if depth_img[i, j] == 0:
                continue

            # 使用内参矩阵的逆计算像素点的3D坐标
            p = np.dot(k_inv, np.array([j, i, 1]))
            p = p * depth_img[i, j]

            # 将3D坐标添加到点云数组中
            point_cloud[i * w + j, :] = p

    return point_cloud


In [21]:
# 十次求平均
# 相机内参矩阵
k = np.array([[535.4, 0.0, 320.0],
              [0.0, 539.2, 240.0],
              [0.0, 0.0, 1.0]])

elapsed_time_list = []
for i in range(10):# 
    start_time = time.time()  # 记录开始时间

    # 将深度图转换为点云
    point_cloud = depth_to_point_cloud(depth_img, k)

    end_time = time.time()  # 记录结束时间

    elapsed_time = end_time - start_time  # 计算耗时

    elapsed_time_list.append(elapsed_time)

average_time = sum(elapsed_time_list) / len(elapsed_time_list)

print(f"Time elapsed average 10 times:{average_time:.4f} seconds.")

Time elapsed:0.9333seconds 


In [17]:
# 可视化点云

# 点云数据存储在名为point_cloud的numpy数组中
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(point_cloud)

# 创建一个可视化窗口
vis = o3d.visualization.Visualizer()
vis.create_window()

# 将点云添加到可视化窗口中
vis.add_geometry(cloud)

# 设置相机位置，没调所以可有可无
vis.get_view_control().set_front([0, 0, -1])
vis.get_view_control().set_lookat([0, 0, 0])
vis.get_view_control().set_up([0, -1, 0])

# 开始显示点云
vis.run()

# 关闭显示窗口
vis.destroy_window()

0.9s